In [1]:
using Oscar
using Combinatorics
pm = Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.10.2 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


Polymake

In [2]:
include("matroid_realization.jl");
include("reduceIdeals.jl")


d3n10 = vec(readlines("d3n10.dat"));


function matroid_to_reduced_expression(Q, F,k)
    charts = [c for c in circuits(Q) if length(c) == 4]
    A = charts[1]
    RQ = matroid_realization_space(Q, A, F)
    R = parent(RQ[1][1])
    Sgens = [s for s in RQ[2] if length(s) <= k]#new 13.1.2023
    I = reduce_ideal_full(RQ[1], Sgens, R, gens(R), false)
    return (I[1], I[2])#I[1] = ideal generators, I[2] = subgroup generators
end



matroid_to_reduced_expression (generic function with 1 method)

In [3]:
d3n10

5249-element Vector{String}:
 "*******************************" ⋯ 58 bytes ⋯ "*******************************"
 "0******************************" ⋯ 58 bytes ⋯ "*******************************"
 "0******************0***********" ⋯ 58 bytes ⋯ "*******************************"
 "0******************0***********" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "****************************0**"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "******************************0"
 "0******0***********************" ⋯ 58 bytes ⋯ "*******************************"
 "0******0***********************" ⋯ 58 bytes ⋯ "*********************0*********"
 "0******0***********************" ⋯ 58 bytes ⋯ "********************

find_lines (generic function with 1 method)

In [15]:
#for an element n, counts how many lines its in
function in_lines(n,L)
    xn = 0
    for z in 1:length(L)
        if n in L[z]
           xn += 1 
        end
    end
    return xn
end

in_lines (generic function with 1 method)

In [16]:
#find simple matroids such that each element is contained in at least 3 lines
#this could be a function that depends on M and k, where k is the number of "lines"
has_property = []
for z in 1:length(d3n10)
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    L = cyclic_flats(Mz,2)
    xm = 0
    for t in 1:10
        if in_lines(t,L)>2
            xm +=1
        end
    end
    #print(z,"\n", xm ,"\n",find_lines(z),"\n")
    if xm == 10
        push!(has_property,z)
    end
end

In [17]:
has_property

151-element Vector{Any}:
  324
  423
  428
  450
  630
  635
  798
  884
  894
  903
  904
  906
  908
    ⋮
 4810
 4822
 4825
 4829
 4839
 4841
 4903
 4910
 4916
 4917
 4919
 5085

In [5]:
#testing code for increasing values of k in matroid_to_reduce_expression
for k in 1:2
    print("\n","\n","k = ",k,"\n","\n")
    Mz = matroid_from_revlex_basis_encoding(d3n10[1072], 3, 10)
    reduced = matroid_to_reduced_expression(Mz, ZZ,k)
    
    for j in 1:length(reduced[1])
    print(j," : ", factor(reduced[1][j]),"\n")
    end
end
#notes for 798:kernal died when I tried 4 and 5, works for 3. Note that for k =2 and k=3 we have the same output. 
#Reduced to something such that we can divide out a monomial to get a degree 2 polynomial in one variable.

#1072 returns cannot localize at 0 element for k = 2. Note that for k=1, 1072 has all variables in degree 1 
#for both generators.




k = 1

1 : 0
2 : -1 * (x_{1, 2}*x_{2, 3}*x_{2, 5}*x_{2, 6} - x_{1, 2}*x_{2, 3}*x_{2, 5} - x_{1, 2}*x_{2, 5}*x_{2, 6} + x_{1, 2}*x_{2, 5} + x_{2, 3}*x_{2, 5} - x_{2, 3}*x_{2, 6})
3 : -1 * (x_{1, 2}*x_{2, 3}*x_{2, 6} - x_{1, 2}*x_{2, 3} - x_{1, 2}*x_{2, 5}*x_{2, 6} + x_{1, 2}*x_{2, 5} + x_{2, 3}*x_{2, 5} - x_{2, 3}*x_{2, 6})


k = 2



LoadError: can not localize at the zero element

In [8]:
function sort(I)
    cc = []
    for f in I[1]
        g =[x for x in vars(f) if degree(f,x) == 1]
        if length(g)>0 
            push!(cc,f)
        end
    end
    if length(cc)>0
        return true
        
        else 
        
        return false
        
    end
end

sort (generic function with 1 method)

In [82]:
sort(reduced)

true

In [ ]:
good = []
more = []
for z in has_property
    print(z)
    Mz = matroid_from_revlex_basis_encoding(d3n10[z], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ,3)
    
    if sort(I)
        print(" in more","\n\n")
        push!(more,z)
    
    else 
        print(" in good","\n\n")
        push!(good,z)
        
    end
end
#testing for higher k. k=2 gives error "can not localize at the zero element" at matroid 1072
#k=3 kills kernal

In [34]:
good

106-element Vector{Any}:
  324
  428
  450
  635
  965
  995
 1061
 1076
 1082
 1252
 1253
 1272
 1273
    ⋮
 4800
 4808
 4809
 4810
 4825
 4829
 4903
 4910
 4916
 4917
 4919
 5085

In [35]:
more

45-element Vector{Any}:
  423
  630
  798
  884
  894
  903
  904
  906
  908
  923
 1070
 1072
 1080
    ⋮
 3772
 3801
 3802
 3811
 3812
 3839
 3840
 4493
 4789
 4822
 4839
 4841

In [100]:
Mz = matroid_from_revlex_basis_encoding(d3n10[3840], 3, 10)
I = matroid_to_reduced_expression(Mz, ZZ)

(fmpz_mpoly[0, 2*x_{1, 6} - 1, 2*x_{1, 6} + 1], fmpz_mpoly[1, -1, x_{1, 6}, -x_{1, 6}])

In [101]:
for i in 1:length(I[1])
    print(i, ": ", I[1][i],"\n")
end

1: 0
2: 2*x_{1, 6} - 1
3: 2*x_{1, 6} + 1


In [102]:
f = I[1][2]
for x in vars(f)
    print(x,":",degree(f,x),"\n")
end

x_{1, 6}:1


In [104]:
int_gen =[]

for x in good
    Mz = matroid_from_revlex_basis_encoding(d3n10[x], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ)
    
    G = [f for f in I[1] if length(vars(f)) == 0]
    
    if length(G)>1
        push!(int_gen,x)
    end
end

In [105]:
int_gen

13-element Vector{Any}:
 1607
 3648
 3658
 3666
 3725
 4800
 4808
 4809
 4810
 4825
 4910
 4917
 4919

In [106]:
for x in int_gen
    
    Mz = matroid_from_revlex_basis_encoding(d3n10[x], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ)
    
    print(x,":", I[1],"\n")
end

1607:fmpz_mpoly[0, 2, -x_{1, 6}^2 + x_{1, 6}*x_{2, 6} + x_{2, 6}^2 - x_{2, 6}, -x_{1, 6}^3 + x_{1, 6}^2*x_{2, 6} - x_{1, 6}*x_{2, 6}^2 + x_{2, 6}^2]
3648:fmpz_mpoly[0, -2, -x_{1, 6}^2 + x_{1, 6} + 1, x_{1, 6}^2 + x_{1, 6} + 1]
3658:fmpz_mpoly[0, 1]
3666:fmpz_mpoly[0, 1, -x_{2, 6}^2 - 1]
3725:fmpz_mpoly[0, 1]
4800:fmpz_mpoly[0, -3]
4808:fmpz_mpoly[0, 3]
4809:fmpz_mpoly[0, 3, -3]
4810:fmpz_mpoly[0, -3, 3]
4825:fmpz_mpoly[0, 1]
4910:fmpz_mpoly[0, -2, -x_{2, 6}^2 + x_{2, 6} - 1]
4917:fmpz_mpoly[0, -2, x_{2, 6}^2 + 1]
4919:fmpz_mpoly[0, 3]


In [107]:
for x in good
    
    Mz = matroid_from_revlex_basis_encoding(d3n10[x], 3, 10)
    I = matroid_to_reduced_expression(Mz, ZZ)
    
    print(x,":", I[1],"\n")
end

324:fmpz_mpoly[0, x_{2, 3}^3*x_{1, 6}^2 - 2*x_{2, 3}^3*x_{1, 6} + x_{2, 3}^3 - x_{2, 3}^2*x_{2, 4}*x_{1, 6}^2 + 3*x_{2, 3}^2*x_{2, 4}*x_{1, 6} - 2*x_{2, 3}^2*x_{2, 4} - 2*x_{2, 3}^2*x_{1, 6}^2 + 2*x_{2, 3}^2*x_{1, 6} - x_{2, 3}*x_{2, 4}^2*x_{1, 6} + x_{2, 3}*x_{2, 4}^2 + x_{2, 3}*x_{2, 4}*x_{1, 6}^2 - 3*x_{2, 3}*x_{2, 4}*x_{1, 6} + x_{2, 3}*x_{2, 4} + x_{2, 3}*x_{1, 6}^2 + x_{2, 4}^2*x_{1, 6} - x_{2, 4}^2]
428:fmpz_mpoly[0, x_{1, 4}^2*x_{2, 6}^2 - 2*x_{1, 4}^2*x_{2, 6} + x_{1, 4}^2 + x_{1, 4}*x_{1, 5}^2*x_{2, 6} - x_{1, 4}*x_{1, 5}*x_{2, 6}^2 + x_{1, 4}*x_{1, 5}*x_{2, 6} - x_{1, 4}*x_{1, 5} - x_{1, 4}*x_{2, 6}^2 + x_{1, 4}*x_{2, 6} - x_{1, 5}^2*x_{2, 6} + x_{1, 5}*x_{2, 6}^2]
450:fmpz_mpoly[0, x_{1, 4}^2*x_{2, 5}*x_{2, 6}^2 - x_{1, 4}^2*x_{2, 5} - 2*x_{1, 4}^2*x_{2, 6}^2 + 2*x_{1, 4}^2*x_{2, 6} - x_{1, 4}*x_{2, 5}^2*x_{2, 6} - x_{1, 4}*x_{2, 5}*x_{2, 6}^2 + 2*x_{1, 4}*x_{2, 5}*x_{2, 6} + x_{1, 4}*x_{2, 6}^2 - x_{1, 4}*x_{2, 6} + x_{2, 5}^2*x_{2, 6} - x_{2, 5}*x_{2, 6}]
635:fmpz_mpoly[0